In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

base_path = '/content/drive/MyDrive/FYDP/ImageData/'
source_folder = os.path.join(base_path, 'all4genreposters_resized')
augment_folder = os.path.join(base_path, 'augmented_new_resized')
os.makedirs(augment_folder)

In [ ]:
import cv2
import numpy as np
# Function to apply brightness alteration to an image
def apply_brightness_alteration(image, brightness_factor):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Scale the V (Value) channel by the brightness factor
    hsv[..., 2] = np.clip(hsv[..., 2] * brightness_factor, 0, 255)

    # Convert the image back to BGR color space
    altered_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return altered_image

# Function to apply color alteration to an image
def apply_color_alteration(image, color_factor):
    # Split the image into its RGB channels
    b, g, r = cv2.split(image)

    # Apply color alteration to each channel
    b = np.clip(b * color_factor, 0, 255)
    g = np.clip(g * color_factor, 0, 255)
    r = np.clip(r * color_factor, 0, 255)

    # Merge the altered channels to get the final image
    altered_image = cv2.merge((b, g, r))

    return altered_image

# Function to apply contrast alteration to an image
def apply_contrast_alteration(image, contrast_factor):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply contrast alteration using the contrast_factor
    adjusted_image = cv2.convertScaleAbs(gray_image, alpha=contrast_factor, beta=0)

    # Convert the adjusted image back to BGR color space
    final_image = cv2.cvtColor(adjusted_image, cv2.COLOR_GRAY2BGR)

    return final_image

# Function to apply sharpness alteration to an image
def apply_sharpness_alteration(image, sharpness_factor):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the grayscale image
    blurred_image = cv2.GaussianBlur(gray_image, (0, 0), sigmaX=sharpness_factor)

    # Calculate the sharpened image as the difference between the original and blurred images
    sharpened_image = cv2.addWeighted(gray_image, 1.5, blurred_image, -0.5, 0)

    # Convert the sharpened image back to BGR color space
    final_image = cv2.cvtColor(sharpened_image, cv2.COLOR_GRAY2BGR)

    return final_image

sharpness_factors = [1.2,  2.5]
brightness_factors = [1.2,  1.5]
contrast_factors = [1.2,  1.6]
color_factors = [0.5, 1.5]

In [ ]:
def augment_image(augment_image_func, factors, image_file_name_parts, image, prefix):
  for idx, item in enumerate(factors):
    temp_image = augment_image_func(image, item)
    temp_image_file_name = f"{image_file_name_parts[0]}_{prefix}_{idx}{image_file_name_parts[-1]}"
    cv2.imwrite(os.path.join(target_path, temp_image_file_name), temp_image)

for genre in os.listdir(source_folder):
  genre_path = os.path.join(source_folder, genre)
  image_files = os.listdir(genre_path)
  target_path = os.path.join(augment_folder, genre)
  os.makedirs(target_path, exist_ok = True)
  for image_file in image_files:
    image_path = os.path.join(genre_path, image_file)
    image = cv2.imread(image_path)
    cv2.imwrite(os.path.join(target_path, image_file), image)
    image_file_name_parts = os.path.splitext(image_file)
    augment_image(apply_sharpness_alteration, sharpness_factors, image_file_name_parts, image, "sharpness")
    augment_image(apply_brightness_alteration, brightness_factors, image_file_name_parts, image, "brightness")
    augment_image(apply_contrast_alteration, contrast_factors, image_file_name_parts, image, "contrast")
    augment_image(apply_color_alteration, color_factors, image_file_name_parts, image, "color")

In [ ]:
#split_code
for genre in os.listdir(augment_folder):
  print(genre, len(os.listdir(os.path.join(augment_folder, genre))))


Action 3033
Romance 2421
Horror 3582
Comedy 2889


In [ ]:
#reduce randomly = 2000 is the limi
import random
import shutil
from sklearn.model_selection import train_test_split


genre_mapping = {'Horror': "0", 'Comedy': "1", 'Romance': "2", 'Action': "3"}

def copy_files(src_path, target_path, files):
  for file in files:
    src_file_path = os.path.join(src_path, file)
    target_file_path = os.path.join(target_path, file)
    shutil.copy(src_file_path, target_file_path)

def prepare_client_files(client_name, genre, genre_path, image_files):
  client_path = os.path.join(base_path, client_name)
  train_path = os.path.join(client_path, 'train')
  val_path = os.path.join(client_path, 'val')
  test_path = os.path.join(client_path, 'test')
  train_genre_path = os.path.join(train_path, genre_mapping.get(genre))
  val_genre_path = os.path.join(val_path, genre_mapping.get(genre))
  test_genre_path = os.path.join(test_path, genre_mapping.get(genre))

  os.makedirs(client_path, exist_ok = True)
  os.makedirs(train_path, exist_ok = True)
  os.makedirs(val_path, exist_ok = True)
  os.makedirs(test_path, exist_ok = True)
  os.makedirs(train_genre_path, exist_ok = True)
  os.makedirs(val_genre_path, exist_ok = True)
  os.makedirs(test_genre_path, exist_ok = True)

  train, val, test = image_files[:700], image_files[700:850], image_files[850:]

  copy_files(genre_path, train_genre_path, train )
  copy_files(genre_path, val_genre_path, val )
  copy_files(genre_path, test_genre_path, test )



for genre in os.listdir(augment_folder):
   genre_path = os.path.join(augment_folder, genre)
   image_files = random.sample(os.listdir(genre_path)[0:2000], 2000)

   client_1_items = image_files[:1000]
   client_2_items = image_files[1000:2000]

   prepare_client_files("fl_client_1", genre, genre_path, client_1_items)
   prepare_client_files("fl_client_2", genre, genre_path, client_2_items)







